In [29]:
import warnings
import firebase_admin
from firebase_admin import credentials, firestore, auth
from collections import defaultdict

warnings.filterwarnings("ignore")

In [ ]:
cred = credentials.Certificate("slts-8e29d-firebase-adminsdk-i6d1u-d3ea7654ac.json")
firebase_admin.initialize_app(cred)
db = firestore.client()

In [31]:
def delete_collection(coll_ref, batch_size):
    if batch_size == 0:
        return

    docs = coll_ref.list_documents(page_size=batch_size)
    deleted = 0

    for doc in docs:
        # print(f"Deleting doc {doc.id} => {doc.get().to_dict()}")
        doc.delete()
        deleted = deleted + 1

    if deleted >= batch_size:
        return delete_collection(coll_ref, batch_size)

# Find users with "judge" in their email and delete them.
for user in auth.list_users().iterate_all():
    if "judge" in user.email:
        auth.delete_user(user.uid)
        # print(f"Deleted user {user.email}")

# Delete all users from "userData" collection whose email contains "judge"
print("Deleting all judges from 'userData' collection...")
users = db.collection("userData").stream()
for user in users:
    if "judge" in user.to_dict()["email"]:
        user.reference.delete()
        # print(f"Deleted user {user.to_dict()['email']}")
print("Successfully deleted all judges from 'userData' collection!")

print("Deleting all event data...")
delete_collection(db.collection("eventData"), 10)
print("Successfully deleted all event data!")

Deleting all judges from 'userData' collection...
Successfully deleted all judges from 'userData' collection!
Deleting all event data...
Successfully deleted all event data!


In [32]:
def addJudge(judgeName, eventName):
    judgeEmail = f"{judgeName}@slts.cbe"
    judgePassword = f"{judgeName}@23111976"
    judge = auth.create_user(email=judgeEmail, password=judgePassword)
    judge_uid = judge.uid

    user_ref = db.collection("userData").document(judge_uid)
    user_ref.set({
        "email": judgeEmail,
        "name": judgeName,
        "id": judge_uid,
        "role": "judge",
        "event": eventName,
        "eventRef": db.collection("eventData").document(eventName),
    })

    event_ref = db.collection("eventData").document(eventName)
    event_ref.update({
        "judgeIdList": firestore.ArrayUnion([judge_uid]),
        "judgeEmailList": firestore.ArrayUnion([judgeEmail]),
    })

In [33]:
group1EventList = [
    "Bhajans",
    "Slokas",
    "Drawing",
    "Story Telling - English",
    "Story Telling - Tamizh",
    "Tamizh Chants",
    "Vedam Chanting",
]

jRefIds = [1, 1, 1, 1]
eventJudgeCount = defaultdict(int)

for event in group1EventList:
    name = f"G1-{event}"
    jRefId = f"g1e{jRefIds[0]}"
    doc_ref = db.collection("eventData").document(name)
    doc_ref.set(
        {
            "name": name,
            "jRefId": jRefId,
            "group": ["Group 1"],
            "evalCriteria": {
                "Criteria 1": 10,
                "Criteria 2": 10,
            },
        }
    )

    for i in range(1, 5):
        addJudge(f"judge{i}{jRefId}", name)
        eventJudgeCount[name] += 1
    
    jRefIds[0] += 1

In [34]:
eventList = [
    "Bhajans - Boys",
    "Bhajans - Girls",
    "Slokas - Boys",
    "Slokas - Girls",
    "Vedam Chanting - Boys",
    "Vedam Chanting - Girls",
    "Elocution - English",
    "Elocution - Tamizh",
    "Drawing",
    "Tamizh Chants - Boys",
    "Tamizh Chants - Girls",
    "GROUP - Altar Decoration - Boys",
    "GROUP - Altar Decoration - Girls",
    "GROUP - Devotional Singing - Boys",
    "GROUP - Devotional Singing - Girls",
    "GROUP - Rudram Namakam Chanting - Boys",
    "GROUP - Rudram Namakam Chanting - Girls",
    "Quiz",
]


for event in eventList:
    if "Devotional" in event:
        jRefId = f"gre{jRefIds[3]}"
        db.collection("eventData").document(event).set(
            {
                "name": event,
                "group": ["Group 1", "Group 2", "Group 3"],
                "jRefId": jRefId,
                "evalCriteria": {
                    "Criteria 1": 10,
                    "Criteria 2": 10,
                },
            }
        )
        
        for i in range(1, 5):
            addJudge(f"judge{i}{jRefId}", event)
            eventJudgeCount[event] += 1

        jRefIds[3] += 1
        continue

    if "GROUP" in event:
        jRefId = f"gre{jRefIds[3]}"
        db.collection("eventData").document(event).set(
            {
                "name": event,
                "group": ["Group 2", "Group 3"],
                "jRefId": jRefId,
                "evalCriteria": {
                    "Criteria 1": 10,
                    "Criteria 2": 10,
                },
            }
        )

        for i in range(1, 5):
            addJudge(f"judge{i}{jRefId}", event)
            eventJudgeCount[event] += 1
        
        jRefIds[3] += 1
        continue

    if "Quiz" in event:
        jRefId = f"gre{jRefIds[3]}"
        db.collection("eventData").document(event).set(
            {
                "name": event,
                "group": ["Group 2", "Group 3", "General Category"],
                "jRefId": jRefId,
                "evalCriteria": {
                    "Criteria 1": 10,
                    "Criteria 2": 10,
                },
            }
        )

        for i in range(1, 5):
            addJudge(f"judge{i}{jRefId}", event)
            eventJudgeCount[event] += 1

        jRefIds[3] += 1
        continue

    name = f"G2-{event}"
    group = "Group 2"
    jRefId = f"g2e{jRefIds[1]}"
    db.collection("eventData").document(name).set(
        {
            "name": name,
            "group": [group],
            "jRefId": jRefId,
            "evalCriteria": {
                "Criteria 1": 10,
                "Criteria 2": 10,
            },
        }
    )

    for i in range(1, 5):
        addJudge(f"judge{i}{jRefId}", name)
        eventJudgeCount[name] += 1
    jRefIds[1] += 1

    name = f"G3-{event}"
    group = "Group 3"
    jRefId = f"g3e{jRefIds[2]}"
    db.collection("eventData").document(name).set(
        {
            "name": name,
            "group": [group],
            "jRefId": jRefId,
            "evalCriteria": {
                "Criteria 1": 10,
                "Criteria 2": 10,
            },
        }
    )

    for i in range(1, 5):
        addJudge(f"judge{i}{jRefId}", name)
        eventJudgeCount[name] += 1
    
    jRefIds[2] += 1


print("Successfully added all judges and event data!")
print("Event Judge Count:")
for event, count in eventJudgeCount.items():
    print(f"{event}: {count}")

Successfully added all judges and event data!
Event Judge Count:
G1-Bhajans: 4
G1-Slokas: 4
G1-Drawing: 4
G1-Story Telling - English: 4
G1-Story Telling - Tamizh: 4
G1-Tamizh Chants: 4
G1-Vedam Chanting: 4
G2-Bhajans - Boys: 4
G3-Bhajans - Boys: 4
G2-Bhajans - Girls: 4
G3-Bhajans - Girls: 4
G2-Slokas - Boys: 4
G3-Slokas - Boys: 4
G2-Slokas - Girls: 4
G3-Slokas - Girls: 4
G2-Vedam Chanting - Boys: 4
G3-Vedam Chanting - Boys: 4
G2-Vedam Chanting - Girls: 4
G3-Vedam Chanting - Girls: 4
G2-Elocution - English: 4
G3-Elocution - English: 4
G2-Elocution - Tamizh: 4
G3-Elocution - Tamizh: 4
G2-Drawing: 4
G3-Drawing: 4
G2-Tamizh Chants - Boys: 4
G3-Tamizh Chants - Boys: 4
G2-Tamizh Chants - Girls: 4
G3-Tamizh Chants - Girls: 4
GROUP - Altar Decoration - Boys: 4
GROUP - Altar Decoration - Girls: 4
GROUP - Devotional Singing - Boys: 4
GROUP - Devotional Singing - Girls: 4
GROUP - Rudram Namakam Chanting - Boys: 4
GROUP - Rudram Namakam Chanting - Girls: 4
Quiz: 4
